In [ ]:
import numpy as np
from scipy.special import iv, ive

def gec_monopore_pdf(t, np_, tp_):
    return iv(1, np.sqrt(4*np_*t/tp_)) * np.sqrt(np_/(t*tp_)) * np.exp(-t/tp_-np_)

def robust_gec_monopore_pdf(t, np_, tp_):
    # Bessel functions in Python that work with large exponents
    # https://stackoverflow.com/questions/13726464/bessel-functions-in-python-that-work-with-large-exponents
    #
    # iv(1, np.sqrt(4*np_*t/tp_)) * np.sqrt(np_/(t*tp_)) * np.exp(-t/tp_-np_)
    #
    # ive(v, z) = iv(v, z) * exp(-abs(z.real))
    # iv(v, sq) = ive(v, sq) * exp(sq)

    # val = single_pore_pdf(t, np_, tp_)
    sq = np.sqrt(4*np_*t/tp_)
    val = ive(1, sq) * np.sqrt(np_/(t*tp_)) * np.exp(sq -t/tp_ -np_)
    isnan_val = np.isnan(val)
    val[isnan_val] = 0
    return val

In [ ]:
t = np.linspace(0.01, 300, 100)
np_ = 100
tp_ = 1
pdf1 = gec_monopore_pdf(t, np_, tp_)
pdf2 = robust_gec_monopore_pdf(t, np_, tp_)
import matplotlib.pyplot as plt
plt.plot(t, pdf1, label='gec_monopore_pdf') 
plt.plot(t, pdf2, label='robust_gec_monopore_pdf', linestyle='dashed')
plt.legend()

# Connecting GEC Monopore PDF to Lévy Process Framework

## Two Equivalent Perspectives on the Same Stochastic Process

### 1. **GEC Approach (Giddings-Eyring-Carmichael 1955)**
The functions above implement the **direct analytical solution** from the original stochastic theory:

```
f(t) = √(np/(t·τp)) · e^(-t/τp - np) · I₁(√(4np·t/τp))
```

**Physical interpretation:**
- `np`: average number of pore ingress/egress cycles
- `τp`: average residence time per pore visit
- Total time in pores: `t = Σ(individual sojourn times)`

**This is a CONVOLUTION APPROACH**: The molecule makes `n` random visits (Poisson distributed), each with exponentially distributed duration `τ`. The total time is the sum of these random variables.

---

### 2. **Lévy Process Approach (Pasti 2005, Dondi 2002)**
The Lévy characteristic function method solves the **same problem** but from a different angle:

```python
# Characteristic function (Fourier domain)
φ(ω) = exp[r̄M * Σ(exp(iω·τS,i) - 1) · ΔF(τS,i)]

# Inverse FFT to get PDF
f(t) = IFFT[φ(ω)]
```

**Physical interpretation:**
- `r̄M`: average number of sorption events (= `np` in GEC)
- `{τS,i, ΔF(τS,i)}`: discrete distribution of sorption times
- For monopore: single value `τS,1 = τp` with `ΔF = 1.0`

**This is a CHARACTERISTIC FUNCTION APPROACH**: Work in Fourier domain where convolutions become multiplications, then transform back.

---

## Key Equivalence

**For monopore (uniform sorption time):**
```
GEC:   np = 100,  τp = 1s
  ↓↓↓ EQUIVALENT ↓↓↓
Lévy:  r̄M = 100,  τS,i = [1s],  ΔF = [1.0]
```

Both give the **same elution peak** near t = 100s!

---

## Why Two Methods?

| Method | Advantage | Use Case |
|--------|-----------|----------|
| **GEC Direct** | Simple closed-form formula | Monopore, analytical calculations |
| **Lévy CF** | Handles complex distributions | Multipore, heterogeneous sites |

**The Lévy approach shines when:**
- Multiple distinct sorption time populations (NS + S sites)
- Continuous distributions F(τS)
- Combining independent processes (multiply CFs!)
- Fitting experimental data to extract sorption time distributions

---

## Connection to Your Session Work

From `SESSION_2024-12-12_levy_process_implementation.md`:
- You implemented `levy_fft_method()` for **general distributions**
- For monopore, it **reduces exactly** to the GEC formula above
- For bipore (NS + S sites), Lévy naturally handles it by CF multiplication

**Next step**: Your monopore functions validate the single-site case. Now you can extend to **multipore SEC** using the Lévy framework!

## Practical Demonstration: Verify Equivalence

In [ ]:
# Lévy approach for monopore (should match GEC functions above)
def levy_monopore_pdf(t, np_, tp_, n_points=8192, return_full_grid=False):
    """
    Monopore PDF using Lévy characteristic function approach.
    This should produce IDENTICAL results to gec_monopore_pdf/robust_gec_monopore_pdf.
    
    CORRECT implementation matching levy_vs_montecarlo_demo.py exactly.
    
    Args:
        t: Time points for output (ignored if return_full_grid=True)
        np_: Mean number of pore cycles
        tp_: Residence time per cycle
        n_points: FFT grid size
        return_full_grid: If True, return (time_grid, pdf) on full FFT grid
    """
    # For monopore: single sorption time value
    tauS_i = np.array([tp_])
    DeltaF_i = np.array([1.0])  # 100% of sites have this sorption time
    rM_bar = np_  # Same as np in GEC notation
    
    # Estimate time scale for grid
    tauS_mean = np.sum(tauS_i * DeltaF_i)
    expected_tS = rM_bar * tauS_mean
    dt = expected_tS / n_points * 4
    
    # Build symmetric frequency array
    omega = np.fft.fftfreq(n_points, dt) * 2 * np.pi
    
    # Characteristic function with EXPONENTIAL distribution (Giddings model)
    # φ(ω) = exp[r̄M * Σ((λ/(λ-iω)) - 1) · ΔF] where λ = 1/τ
    # NOT delta function: exp[r̄M * Σ(exp(iω·τ) - 1) · ΔF] ← WRONG!
    sum_term = np.zeros_like(omega, dtype=complex)
    for tau, deltaF in zip(tauS_i, DeltaF_i):
        lambda_exp = 1.0 / tau
        phi_X = lambda_exp / (lambda_exp - 1j * omega)  # Exponential CF
        sum_term += (phi_X - 1) * deltaF
    cf = np.exp(rM_bar * sum_term)
    
    # Inverse FFT to get PDF
    peak = np.fft.ifft(cf).real
    
    # Shift so time starts at 0 (FFT assumes periodic, centered at 0)
    peak = np.fft.ifftshift(peak)
    
    # Ensure non-negative BEFORE normalizing (clipping changes the sum!)
    peak = np.maximum(peak, 0)
    
    # Normalize to unit area AFTER clipping
    peak = peak / (np.sum(peak) * dt)
    
    # Time grid: simple sequential array starting from 0
    time_grid = np.arange(n_points) * dt
    
    if return_full_grid:
        return time_grid, peak
    else:
        # Interpolate to match input time array
        pdf_levy = np.interp(t, time_grid, peak, left=0, right=0)
        return pdf_levy

# Test equivalence
pdf_gec = robust_gec_monopore_pdf(t, np_, tp_)
pdf_levy = levy_monopore_pdf(t, np_, tp_)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(t, pdf_gec, 'b-', linewidth=2, label='GEC (Bessel function)', alpha=0.7)
plt.plot(t, pdf_levy, 'r--', linewidth=2, label='Lévy (CF + FFT)', alpha=0.7)
plt.xlabel('Time (s)')
plt.ylabel('Probability Density')
plt.title('Monopore PDF: GEC vs Lévy Approach')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(t, pdf_gec - pdf_levy, 'k-', linewidth=1.5)
plt.axhline(0, color='r', linestyle='--', alpha=0.5)
plt.xlabel('Time (s)')
plt.ylabel('Difference (GEC - Lévy)')
plt.title('Residuals (should be ~0)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Maximum difference: {np.max(np.abs(pdf_gec - pdf_levy)):.2e}")
print(f"Mean squared error: {np.mean((pdf_gec - pdf_levy)**2):.2e}")
print("\n✓ Both methods are mathematically equivalent!")

### Diagnostic Check: Visualize the full Lévy FFT output

Let's see what the Lévy method produces before interpolation:

In [ ]:
# Let's create the full Lévy distribution to see what's happening
n_points_diag = 8192
tauS_mean = tp_
expected_tS = np_ * tauS_mean
dt_diag = expected_tS / n_points_diag * 4  # Match the function

omega_diag = np.fft.fftfreq(n_points_diag, dt_diag) * 2 * np.pi

# Use loop like in levy_vs_montecarlo_demo.py
sum_term_diag = np.zeros_like(omega_diag, dtype=complex)
for tau, deltaF in zip([tp_], [1.0]):
    sum_term_diag += (np.exp(1j * omega_diag * tau) - 1) * deltaF
cf_diag = np.exp(np_ * sum_term_diag)

peak_fft_diag = np.fft.ifft(cf_diag).real
peak_shifted_diag = np.fft.ifftshift(peak_fft_diag)

# Simple time grid starting from 0
time_grid_diag = np.arange(n_points_diag) * dt_diag

# Normalize
peak_shifted_diag = peak_shifted_diag / (np.sum(peak_shifted_diag) * dt_diag)
peak_shifted_diag = np.maximum(peak_shifted_diag, 0)

# Compare on full grid
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(time_grid_diag, peak_shifted_diag, 'r-', linewidth=1.5, label='Lévy (full grid)', alpha=0.8)
t_dense = np.linspace(0.01, 300, 1000)
pdf_gec_dense = robust_gec_monopore_pdf(t_dense, np_, tp_)
plt.plot(t_dense, pdf_gec_dense, 'b--', linewidth=1.5, label='GEC reference', alpha=0.8)
plt.xlabel('Time (s)')
plt.ylabel('Probability Density')
plt.title('Full Grid Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 200)

plt.subplot(1, 2, 2)
plt.plot(time_grid_diag, peak_shifted_diag, 'r-', linewidth=1.5, label='Lévy', alpha=0.8)
plt.plot(t_dense, pdf_gec_dense, 'b--', linewidth=1.5, label='GEC', alpha=0.8)
plt.xlabel('Time (s)')
plt.ylabel('Probability Density')
plt.title('Zoom: Peak Region')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(80, 120)
plt.ylim(0, max(pdf_gec_dense.max(), peak_shifted_diag.max()) * 1.1)

plt.tight_layout()
plt.show()

print(f"Lévy peak location: {time_grid_diag[np.argmax(peak_shifted_diag)]:.2f} s")
print(f"GEC peak location: {t_dense[np.argmax(pdf_gec_dense)]:.2f} s")
print(f"Expected (theory): {np_ * tp_} s")

In [ ]:
# Additional verification: Compare moments
# The moments should match exactly if the methods are equivalent

# Calculate moments for GEC on the interpolated grid
mean_gec = np.trapezoid(t * pdf_gec, t)
var_gec = np.trapezoid((t - mean_gec)**2 * pdf_gec, t)
std_gec = np.sqrt(var_gec)

# For Lévy, calculate moments on the FULL FFT grid (before interpolation)
# This is CRITICAL - moments computed on interpolated grid are wrong!
time_levy_full, pdf_levy_full = levy_monopore_pdf(t, np_, tp_, return_full_grid=True)
dt_levy = time_levy_full[1] - time_levy_full[0]

# Debug: Check FFT grid coverage
print(f"DEBUG: FFT grid extends from {time_levy_full[0]:.2f} to {time_levy_full[-1]:.2f} s")
print(f"DEBUG: dt = {dt_levy:.6f}, n_points = {len(time_levy_full)}")
print(f"DEBUG: Peak at index {np.argmax(pdf_levy_full)} → time {time_levy_full[np.argmax(pdf_levy_full)]:.2f} s")
print(f"DEBUG: PDF array sum = {np.sum(pdf_levy_full):.6f}")
print(f"DEBUG: PDF integral (sum * dt) = {np.sum(pdf_levy_full * dt_levy):.6f} (should be ~1.0)")
print(f"DEBUG: max(pdf) = {np.max(pdf_levy_full):.6f}")
print()

mean_levy = np.sum(time_levy_full * pdf_levy_full * dt_levy)
var_levy = np.sum((time_levy_full - mean_levy)**2 * pdf_levy_full * dt_levy)
std_levy = np.sqrt(var_levy)

# Theoretical predictions from stochastic theory (Giddings 1955, eqs 12-13)
mean_theory = np_ * tp_  # First moment: μ₁ = n̄p·τp
var_theory = 2 * np_ * tp_**2  # Second central moment: μ'₂ = 2·n̄p·τp²
std_theory = np.sqrt(var_theory)

print("Statistical Moments Comparison:")
print("=" * 60)
print(f"{'Method':<15} {'Mean':<12} {'Std Dev':<12} {'Variance':<12}")
print("-" * 60)
print(f"{'GEC':<15} {mean_gec:<12.4f} {std_gec:<12.4f} {var_gec:<12.4f}")
print(f"{'Lévy':<15} {mean_levy:<12.4f} {std_levy:<12.4f} {var_levy:<12.4f}")
print(f"{'Theory':<15} {mean_theory:<12.4f} {std_theory:<12.4f} {var_theory:<12.4f}")
print("-" * 60)
print(f"{'GEC vs Lévy':<15} {abs(mean_gec-mean_levy):<12.2e} {abs(std_gec-std_levy):<12.2e} {abs(var_gec-var_levy):<12.2e}")
print(f"{'GEC vs Theory':<15} {abs(mean_gec-mean_theory):<12.2e} {abs(std_gec-std_theory):<12.2e} {abs(var_gec-var_theory):<12.2e}")
print("=" * 60)

## Why This Matters for SEC-SAXS

From the papers you read:

### **1955 Giddings Paper:**
- **Original context**: Adsorption chromatography with rate constants k_i (adsorption) and k'_i (desorption)
- **Key insight**: "Dispersion is proportional to √(flow rate)" - random walk mechanism
- **Peak shape**: Modified Bessel function I₁, **NOT Gaussian** but shows positive skew

### **2014 Sepsey Paper (SEC-specific):**
- **SEC mechanism**: Size exclusion, not adsorption
- **Pore size distribution**: Real columns have lognormal PSD (σ parameter)
- **Two processes**: 
  - Pore ingress (characterized by m_e)
  - Pore egress (characterized by m_p)
- **Key finding**: "Wide PSD increases retention and efficiency for macromolecules"

### **Connection to Your Lévy Work:**
The Lévy framework from your Dec 12 session provides the **computational machinery** to:

1. **Handle distributions**: Not just monopore, but arbitrary {τS,i, ΔF(τS,i)}
2. **Combine processes**: NS sites × S sites via CF multiplication
3. **Fit data**: Extract sorption time distributions from experimental elution curves

### **For molass SDM re-implementation:**
```python
# Current approach (empirical):
params = estimate_sdm_column_params()  # Fit moments

# Proposed Lévy approach (physical):
{τS,i, ΔF_i} = extract_sorption_distribution(elution_curve)
peak = levy_fft_method(τS_i, ΔF_i, rM_bar)
```

**Advantage**: The sorption time distribution has **physical meaning** - you can see which molecular populations contribute to dispersion!

## Alternative Approach: Using Legacy molass FFT Implementation

Let's try the FFT implementation from the legacy codebase to see if it handles normalization differently:

In [ ]:
# Import legacy FFT implementation
from molass_legacy.SecTheory.SecPDF import FftInvPdf, FftInvImpl

# Define the Lévy characteristic function for monopore (CORRECT exponential version)
def levy_monopore_cf(omega, np_, tp_):
    """
    Characteristic function for monopore Lévy process with EXPONENTIAL distribution.
    φ(ω) = exp[np * ((λ/(λ-iω)) - 1)] where λ = 1/tp
    NOT delta function: exp[np * (exp(iω*tp) - 1)] ← WRONG!
    """
    lambda_exp = 1.0 / tp_
    phi_X = lambda_exp / (lambda_exp - 1j * omega)
    return np.exp(np_ * (phi_X - 1))

# Method 1: Using FftInvPdf class
fft_inv = FftInvPdf(levy_monopore_cf)
pdf_legacy1 = fft_inv(t, np_, tp_)

# Method 2: Using FftInvImpl with pre-computed CF values
fft_impl = FftInvImpl()
omega_legacy = fft_impl.get_w()
cf_values = levy_monopore_cf(omega_legacy, np_, tp_)
pdf_legacy2 = fft_impl.compute(t, cf_values)

# Compare all three approaches
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(t, pdf_gec, 'b-', linewidth=2, label='GEC (Bessel)', alpha=0.7)
plt.plot(t, pdf_levy, 'r--', linewidth=2, label='Lévy (our FFT)', alpha=0.7)
plt.plot(t, pdf_legacy1, 'g:', linewidth=2, label='Legacy FFT (method 1)', alpha=0.7)
plt.xlabel('Time (s)')
plt.ylabel('Probability Density')
plt.title('PDF Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
plt.plot(t, pdf_gec, 'b-', linewidth=2, label='GEC', alpha=0.7)
plt.plot(t, pdf_legacy2, 'm-.', linewidth=2, label='Legacy FFT (method 2)', alpha=0.7)
plt.xlabel('Time (s)')
plt.ylabel('Probability Density')
plt.title('GEC vs Legacy FFT (method 2)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
plt.plot(t, pdf_gec - pdf_levy, 'r-', linewidth=1.5, label='GEC - Our Lévy', alpha=0.7)
plt.plot(t, pdf_gec - pdf_legacy1, 'g-', linewidth=1.5, label='GEC - Legacy1', alpha=0.7)
plt.plot(t, pdf_gec - pdf_legacy2, 'm-', linewidth=1.5, label='GEC - Legacy2', alpha=0.7)
plt.axhline(0, color='k', linestyle='--', alpha=0.3)
plt.xlabel('Time (s)')
plt.ylabel('Residuals')
plt.title('Residuals Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Our Lévy max error:    {np.max(np.abs(pdf_gec - pdf_levy)):.2e}")
print(f"Legacy1 max error:     {np.max(np.abs(pdf_gec - pdf_legacy1)):.2e}")
print(f"Legacy2 max error:     {np.max(np.abs(pdf_gec - pdf_legacy2)):.2e}")

In [ ]:
# Compute moments for legacy FFT approaches
mean_legacy1 = np.trapezoid(t * pdf_legacy1, t)
var_legacy1 = np.trapezoid((t - mean_legacy1)**2 * pdf_legacy1, t)
std_legacy1 = np.sqrt(var_legacy1)

mean_legacy2 = np.trapezoid(t * pdf_legacy2, t)
var_legacy2 = np.trapezoid((t - mean_legacy2)**2 * pdf_legacy2, t)
std_legacy2 = np.sqrt(var_legacy2)

print("Statistical Moments Comparison (Including Legacy FFT):")
print("=" * 70)
print(f"{'Method':<20} {'Mean':<12} {'Std Dev':<12} {'Variance':<12}")
print("-" * 70)
print(f"{'GEC':<20} {mean_gec:<12.4f} {std_gec:<12.4f} {var_gec:<12.4f}")
print(f"{'Our Lévy FFT':<20} {mean_levy:<12.4f} {std_levy:<12.4f} {var_levy:<12.4f}")
print(f"{'Legacy FFT (v1)':<20} {mean_legacy1:<12.4f} {std_legacy1:<12.4f} {var_legacy1:<12.4f}")
print(f"{'Legacy FFT (v2)':<20} {mean_legacy2:<12.4f} {std_legacy2:<12.4f} {var_legacy2:<12.4f}")
print(f"{'Theory':<20} {mean_theory:<12.4f} {std_theory:<12.4f} {var_theory:<12.4f}")
print("-" * 70)
print(f"{'GEC vs Our Lévy':<20} {abs(mean_gec-mean_levy):<12.2e} {abs(std_gec-std_levy):<12.2e} {abs(var_gec-var_levy):<12.2e}")
print(f"{'GEC vs Legacy1':<20} {abs(mean_gec-mean_legacy1):<12.2e} {abs(std_gec-std_legacy1):<12.2e} {abs(var_gec-var_legacy1):<12.2e}")
print(f"{'GEC vs Legacy2':<20} {abs(mean_gec-mean_legacy2):<12.2e} {abs(std_gec-std_legacy2):<12.2e} {abs(var_gec-var_legacy2):<12.2e}")
print("=" * 70)

## Investigating the Variance Issue

Both our FFT and the legacy FFT give variance = 100 instead of the theoretical 200.
Let's investigate why this factor of 2 is missing.

In [ ]:
# First, let's verify the moments from the Lévy CF analytically
# For φ(ω) = exp[np * (exp(iω*τp) - 1)], we can compute moments via derivatives

# The characteristic function for a compound Poisson process is:
# φ(ω) = exp[λ * (φ_X(ω) - 1)]
# where λ is the rate (np in our case), and φ_X is the CF of individual events

# For exponential distribution with rate parameter λ=1/τp:
# φ_X(ω) = λ/(λ - iω) = (1/τp) / (1/τp - iω)

# But our formula uses: exp(iω*τp) which is the CF of a DELTA function at τp, not exponential!

# Let's compute what variance we SHOULD get from exp[np * (exp(iω*τp) - 1)]
omega_test = np.linspace(-0.1, 0.1, 1000)

# Our current CF
cf_delta = np.exp(np_ * (np.exp(1j * omega_test * tp_) - 1))

# Taylor expand to get moments
# φ(ω) ≈ exp[np * (1 + iω·τp + (iω·τp)²/2 - 1)] = exp[np * (iω·τp + (iω·τp)²/2)]
# = exp[i·np·τp·ω - np·τp²·ω²/2]
# This is CF of normal with mean=np·τp, variance=np·τp²

print("Variance from CF φ(ω) = exp[np * (exp(iω*τp) - 1)]:")
print(f"  Taylor expansion gives: Var = np·τp² = {np_ * tp_**2}")
print(f"  But Giddings theory says: Var = 2·np·τp² = {2 * np_ * tp_**2}")
print()

# The issue: We're using a DELTA function (deterministic τp) instead of EXPONENTIAL
# For exponential with mean τp, the CF is: (1/τp)/(1/τp - iω)
# Let's try using the correct CF for exponential distribution

def levy_cf_exponential(omega, np_, tp_):
    """
    Characteristic function using EXPONENTIAL distribution for individual events.
    Each event has exponential distribution with mean τp.
    """
    lambda_exp = 1.0 / tp_
    phi_X = lambda_exp / (lambda_exp - 1j * omega)  # CF of exponential
    return np.exp(np_ * (phi_X - 1))

cf_exponential = levy_cf_exponential(omega_test, np_, tp_)

# Compare the two
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(omega_test, cf_delta.real, 'r-', label='Delta (current)', linewidth=2)
plt.plot(omega_test, cf_exponential.real, 'b--', label='Exponential (correct)', linewidth=2)
plt.xlabel('ω')
plt.ylabel('Re[φ(ω)]')
plt.title('Real part of CF')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(omega_test, cf_delta.imag, 'r-', label='Delta (current)', linewidth=2)
plt.plot(omega_test, cf_exponential.imag, 'b--', label='Exponential (correct)', linewidth=2)
plt.xlabel('ω')
plt.ylabel('Im[φ(ω)]')
plt.title('Imaginary part of CF')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Theoretical variance for exponential individual events:")
print(f"  Mean of exponential: E[X] = τp = {tp_}")
print(f"  Variance of exponential: Var[X] = τp² = {tp_**2}")
print(f"  Second moment: E[X²] = Var[X] + E[X]² = 2τp² = {2*tp_**2}")
print(f"  Compound Poisson variance: np·E[X²] = np·2τp² = {np_ * 2 * tp_**2}")

In [ ]:
# Now let's compute the PDF using the CORRECT exponential CF
def levy_monopore_pdf_exponential(t, np_, tp_, n_points=8192, return_full_grid=False):
    """
    Monopore PDF using Lévy CF with EXPONENTIAL distribution for individual events.
    This should give the correct variance = 2·np·τp².
    """
    # Estimate time scale for grid
    expected_mean = np_ * tp_
    dt = expected_mean / n_points * 4
    
    # Build frequency array
    omega = np.fft.fftfreq(n_points, dt) * 2 * np.pi
    
    # Characteristic function with EXPONENTIAL individual events
    lambda_exp = 1.0 / tp_
    phi_X = lambda_exp / (lambda_exp - 1j * omega)  # CF of exponential(τp)
    cf = np.exp(np_ * (phi_X - 1))
    
    # IFFT to get PDF
    peak = np.fft.ifft(cf).real
    peak = np.fft.ifftshift(peak)
    
    # Ensure non-negative BEFORE normalizing
    peak = np.maximum(peak, 0)
    
    # Normalize to unit area AFTER clipping
    peak = peak / (np.sum(peak) * dt)
    
    # Time grid
    time_grid = np.arange(n_points) * dt
    
    if return_full_grid:
        return time_grid, peak
    else:
        pdf = np.interp(t, time_grid, peak, left=0, right=0)
        return pdf

# Test the exponential CF approach
pdf_levy_exp = levy_monopore_pdf_exponential(t, np_, tp_)

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(t, pdf_gec, 'b-', linewidth=2, label='GEC (Bessel)', alpha=0.7)
# plt.plot(t, pdf_levy, 'r--', linewidth=2, label='Lévy (delta CF)', alpha=0.7)
plt.plot(t, pdf_levy_exp, 'g:', linewidth=2, label='Lévy (exponential CF)', alpha=0.7)
plt.xlabel('Time (s)')
plt.ylabel('Probability Density')
plt.title('PDF Comparison: Delta vs Exponential CF')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# plt.plot(t, pdf_gec - pdf_levy, 'r-', linewidth=1.5, label='GEC - Delta CF', alpha=0.7)
plt.plot(t, pdf_gec - pdf_levy_exp, 'g-', linewidth=1.5, label='GEC - Exponential CF', alpha=0.7)
plt.axhline(0, color='k', linestyle='--', alpha=0.3)
plt.xlabel('Time (s)')
plt.ylabel('Residuals')
plt.title('Residuals')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Delta CF max error:       {np.max(np.abs(pdf_gec - pdf_levy)):.2e}")
print(f"Exponential CF max error: {np.max(np.abs(pdf_gec - pdf_levy_exp)):.2e}")

In [ ]:
# Compute moments for exponential CF approach
time_levy_exp_full, pdf_levy_exp_full = levy_monopore_pdf_exponential(t, np_, tp_, return_full_grid=True)
dt_levy_exp = time_levy_exp_full[1] - time_levy_exp_full[0]

mean_levy_exp = np.sum(time_levy_exp_full * pdf_levy_exp_full * dt_levy_exp)
var_levy_exp = np.sum((time_levy_exp_full - mean_levy_exp)**2 * pdf_levy_exp_full * dt_levy_exp)
std_levy_exp = np.sqrt(var_levy_exp)

print("Statistical Moments Comparison:")
print("=" * 80)
print(f"{'Method':<25} {'Mean':<12} {'Std Dev':<12} {'Variance':<12}")
print("-" * 80)
print(f"{'GEC (Bessel)':<25} {mean_gec:<12.4f} {std_gec:<12.4f} {var_gec:<12.4f}")
print(f"{'Lévy (delta CF)':<25} {mean_levy:<12.4f} {std_levy:<12.4f} {var_levy:<12.4f}")
print(f"{'Lévy (exponential CF)':<25} {mean_levy_exp:<12.4f} {std_levy_exp:<12.4f} {var_levy_exp:<12.4f}")
print(f"{'Legacy FFT (delta CF)':<25} {mean_legacy1:<12.4f} {std_legacy1:<12.4f} {var_legacy1:<12.4f}")
print(f"{'Theory':<25} {mean_theory:<12.4f} {std_theory:<12.4f} {var_theory:<12.4f}")
print("-" * 80)
print(f"{'GEC vs Lévy (delta)':<25} {abs(mean_gec-mean_levy):<12.2e} {abs(std_gec-std_levy):<12.2e} {abs(var_gec-var_levy):<12.2e}")
print(f"{'GEC vs Lévy (exponential)':<25} {abs(mean_gec-mean_levy_exp):<12.2e} {abs(std_gec-std_levy_exp):<12.2e} {abs(var_gec-var_levy_exp):<12.2e}")
print(f"{'GEC vs Theory':<25} {abs(mean_gec-mean_theory):<12.2e} {abs(std_gec-std_theory):<12.2e} {abs(var_gec-var_theory):<12.2e}")
print("=" * 80)

if abs(var_levy_exp - var_theory) < 1.0:
    print("\n✓✓✓ SUCCESS! Exponential CF gives the correct variance!")
else:
    print(f"\n✗ Variance still wrong: {var_levy_exp:.2f} vs {var_theory:.2f}")